# Saliency metrics

In [57]:
import numpy as np
import matplotlib.pyplot as plt
import random
import sys
import cv2
import math

def generate_dummy(size=14,num_fixations=100,num_salience_points=200):
	# first generate dummy gt and salience map
	discrete_gt = np.zeros((size,size))
	s_map = np.zeros((size,size))

	for i in range(0,num_fixations):
		discrete_gt[np.random.randint(size),np.random.randint(size)] = 1.0

	for i in range(0,num_salience_points):
		s_map[np.random.randint(size),np.random.randint(size)] = 255*round(random.random(),1)


	# check if gt and s_map are same size
	assert discrete_gt.shape==s_map.shape, 'sizes of ground truth and salience map don\'t match'
	return s_map,discrete_gt


def normalize_map(s_map):
	# normalize the salience map (as done in MIT code)
	norm_s_map = (s_map - np.min(s_map))/((np.max(s_map)-np.min(s_map))*1.0)
	return norm_s_map

def discretize_gt(gt):
	# import warnings
	# warnings.warn('can improve the way GT is discretized')
	return gt/255

def auc_judd(s_map,gt):
	# ground truth is discrete, s_map is continous and normalized
	gt = discretize_gt(gt)
	# thresholds are calculated from the salience map, only at places where fixations are present
	thresholds = []
	for i in range(0,gt.shape[0]):
		for k in range(0,gt.shape[1]):
			if gt[i][k]>0:
				thresholds.append(s_map[i][k])

	
	num_fixations = np.sum(gt)
	# num fixations is no. of salience map values at gt >0


	thresholds = sorted(set(thresholds))
	
	#fp_list = []
	#tp_list = []
	area = []
	area.append((0.0,0.0))
	for thresh in thresholds:
		# in the salience map, keep only those pixels with values above threshold
		temp = np.zeros(s_map.shape)
		temp[s_map>=thresh] = 1.0
		assert np.max(gt)==1.0, 'something is wrong with ground truth..not discretized properly max value > 1.0'
		assert np.max(s_map)==1.0, 'something is wrong with salience map..not normalized properly max value > 1.0'
		num_overlap = np.where(np.add(temp,gt)==2)[0].shape[0]
		tp = num_overlap/(num_fixations*1.0)
		
		# total number of pixels > threshold - number of pixels that overlap with gt / total number of non fixated pixels
		# this becomes nan when gt is full of fixations..this won't happen
		fp = (np.sum(temp) - num_overlap)/((np.shape(gt)[0] * np.shape(gt)[1]) - num_fixations)
		
		area.append((round(tp,4),round(fp,4)))
		#tp_list.append(tp)
		#fp_list.append(fp)

	#tp_list.reverse()
	#fp_list.reverse()
	area.append((1.0,1.0))
	#tp_list.append(1.0)
	#fp_list.append(1.0)
	#print tp_list
	area.sort(key = lambda x:x[0])
	tp_list =  [x[0] for x in area]
	fp_list =  [x[1] for x in area]
	return np.trapz(np.array(tp_list),np.array(fp_list))



def nss(s_map,gt):
	gt = discretize_gt(gt)
	s_map_norm = (s_map - np.mean(s_map))/np.std(s_map)

	x,y = np.where(gt==1)
	temp = []
	for i in zip(x,y):
		temp.append(s_map_norm[i[0],i[1]])
	return np.mean(temp)


def infogain(s_map,gt,baseline_map):
	gt = discretize_gt(gt)
	# assuming s_map and baseline_map are normalized
	eps = 2.2204e-16

	s_map = s_map/(np.sum(s_map)*1.0)
	baseline_map = baseline_map/(np.sum(baseline_map)*1.0)

	# for all places where gt=1, calculate info gain
	temp = []
	x,y = np.where(gt==1)
	for i in zip(x,y):
		temp.append(np.log2(eps + s_map[i[0],i[1]]) - np.log2(eps + baseline_map[i[0],i[1]]))

	return np.mean(temp)



def similarity(s_map,gt):
	# here gt is not discretized nor normalized
	s_map = normalize_map(s_map)
	gt = normalize_map(gt)
	s_map = s_map/(np.sum(s_map)*1.0)
	gt = gt/(np.sum(gt)*1.0)
	x,y = np.where(gt>0)
	sim = 0.0
	for i in zip(x,y):
		sim = sim + min(gt[i[0],i[1]],s_map[i[0],i[1]])
	return sim


def cc(s_map,gt):
	s_map_norm = (s_map - np.mean(s_map))/np.std(s_map)
	gt_norm = (gt - np.mean(gt))/np.std(gt)
	a = s_map_norm
	b= gt_norm
	r = (a*b).sum() / math.sqrt((a*a).sum() * (b*b).sum());
	return r


def kldiv(s_map,gt):
	s_map = s_map/(np.sum(s_map)*1.0)
	gt = gt/(np.sum(gt)*1.0)
	eps = 2.2204e-16
	return np.sum(gt * np.log(eps + gt/(s_map + eps)))


def calc_metrics(pred_sal, pred_fix, gt_sal, gt_fix ):  
    metrics = {
        'KLDiv' : np.nan, 
        'CC' : np.nan,
        'SIM' : np.nan, 
        'AuC' : np.nan, 
        'NSS' : np.nan, 
        'InfoGain' : np.nan
    }
    try : 
        metrics['KLDiv']  = kldiv(pred_sal,gt_sal)
    except: 
        metrics['KLDiv']  = np.nan

    try : 
        metrics['CC']  = cc(pred_sal,gt_sal)
    except: 
        metrics['CC']  = np.nan

    try : 
        metrics['AuC']  = auc_judd(pred_sal,gt_sal)
    except: 
        metrics['AuC']  = np.nan


    try : 
        metrics['SIM']  = similarity(pred_sal,gt_sal)
    except: 
        metrics['SIM']  = np.nan

    try : 
        metrics['NSS']  = nss(pred_sal,gt_sal)
    except: 
        metrics['NSS']  = np.nan

    try : 
        metrics['InfoGain']  = infogain(pred_sal,gt_sal)
    except: 
        metrics['InfoGain']  = np.nan

    
    return metrics



In [ ]:
import pandas as pd 
import os 
from tqdm.notebook import tqdm 

salmap_folder = "./salmaps/" 
fixation_folder = "./fix_maps/"

td_folder = "TD"
tdex_folder = "TDex"
tsa_folder = "TSA"

stimuli_names = os.listdir(os.path.join(fixation_folder, norm_folder)) 

metrics_names = ['KLDiv','CC' , 'SIM','AuC','NSS', 'InfoGain'] 

metrics_TD_TSA = pd.DataFrame(columns = metrics_names)
metrics_TDex_TSA = pd.DataFrame(columns = metrics_names)

# metrics_TD_TSA = pd.concat([metrics ,  d],  ) 
# metrics.append(d)

# Compare TSA <=> TD 
for stimulus_name  in tqdm(stimuli_names):
    print(stimulus_name)
    tsa_fix_map_p_f = os.path.join(fixation_folder,tsa_folder,stimulus_name)
    tsa_sal_map_p_f = os.path.join(salmap_folder,tsa_folder,stimulus_name)
    td_fix_map_p_f = os.path.join(fixation_folder,td_folder,stimulus_name)
    td_sal_map_p_f = os.path.join(salmap_folder,td_folder,stimulus_name)

    tsa_fix_map_p = os.path.join(tsa_fix_map_p_f, [ x for x  in  os.listdir(tsa_fix_map_p_f) if not ("ipy" in  x) ][0]) 
    tsa_sal_map_p = os.path.join(tsa_sal_map_p_f, [ x for x  in  os.listdir(tsa_sal_map_p_f) if not ("ipy" in  x)][0] )
    td_fix_map_p = os.path.join(td_fix_map_p_f, [ x for x  in  os.listdir(td_fix_map_p_f)  if not ("ipy" in  x) ][0] )
    td_sal_map_p = os.path.join(td_sal_map_p_f, [ x for x  in  os.listdir(td_sal_map_p_f) if not ("ipy" in  x)][0])

    tsa_fix_map = cv2.imread( tsa_fix_map_p ,0)  
    tsa_sal_map = cv2.imread(tsa_sal_map_p ,0)
    td_fix_map = cv2.imread(td_fix_map_p ,0)
    td_sal_map = cv2.imread( td_sal_map_p,0)

    # print(tsa_fix_map.shape , tsa_sal_map.shape,  td_fix_map.shape,  td_sal_map.shape)
    metrics_row = calc_metrics(tsa_sal_map, tsa_fix_map, td_fix_map,td_sal_map)  
    metrics_row = pd.DataFrame([metrics_row])
    metrics_TD_TSA = pd.concat([metrics_TD_TSA,metrics_row], ignore_index=True)  
     

  0%|          | 0/34 [00:00<?, ?it/s]

c1_o2_civita.jpeg


/tmp/ipykernel_923639/3457132532.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_TD_TSA = pd.concat([metrics_TD_TSA,metrics_row] , ignore_index=True)


c3_o6_three.jpeg


In [60]:

# metrics_TD_TSA = pd.concat([metrics ,  d],  ) 
# metrics.append(d)

# Compare TSA <=> TDex 
for stimulus_name  in tqdm(stimuli_names):
    print(stimulus_name)
    td_fix_map_p_f = os.path.join(fixation_folder,td_folder,stimulus_name)
    td_sal_map_p_f = os.path.join(salmap_folder,td_folder,stimulus_name)
    tdex_fix_map_p_f = os.path.join(fixation_folder,tdex_folder,stimulus_name)
    tdex_sal_map_p_f = os.path.join(salmap_folder,tdex_folder,stimulus_name)

    td_fix_map_p = os.path.join(td_fix_map_p_f, [ x for x  in  os.listdir(td_fix_map_p_f) if not ("ipy" in  x) ][0]) 
    td_sal_map_p = os.path.join(td_sal_map_p_f, [ x for x  in  os.listdir(td_sal_map_p_f) if not ("ipy" in  x)][0] )
    tdex_fix_map_p = os.path.join(tdex_fix_map_p_f, [ x for x  in  os.listdir(tdex_fix_map_p_f)  if not ("ipy" in  x) ][0] )
    tdex_sal_map_p = os.path.join(tdex_sal_map_p_f, [ x for x  in  os.listdir(tdex_sal_map_p_f) if not ("ipy" in  x)][0])

    td_fix_map = cv2.imread( td_fix_map_p ,0)  
    td_sal_map = cv2.imread(td_sal_map_p ,0)
    tdex_fix_map = cv2.imread(tdex_fix_map_p ,0)
    tdex_sal_map = cv2.imread( tdex_sal_map_p,0)

    # print(tsa_fix_map.shape , tsa_sal_map.shape,  td_fix_map.shape,  td_sal_map.shape)
    metrics_row = calc_metrics(td_sal_map, td_fix_map, tdex_fix_map, tdex_sal_map)  
    metrics_row = pd.DataFrame([metrics_row])
    metrics_TDex_TSA = pd.concat([metrics_TDex_TSA,metrics_row], ignore_index=True)  
     

  0%|          | 0/34 [00:00<?, ?it/s]

c1_o2_civita.jpeg


/tmp/ipykernel_923639/516314851.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_TDex_TSA = pd.concat([metrics_TDex_TSA,metrics_row], ignore_index=True)


c3_o6_three.jpeg
c4_o4_woman.jpeg
sam_beauté_jpeg.jpeg
c5_o5_field.jpeg
c4_o5_ventimiglia.jpeg
c2_o2_stora.jpeg
c4_o3_persistance.jpeg
c3_o5_antigraceful.jpeg
c5_o4_sails.jpeg
c4_o6_monde.jpeg
connaissance_jpeg.jpeg
c2_o1_lordship.jpeg
c5_o2_based.jpeg
c1_o6_embouchment.jpeg
c2_o4_saint_brac.jpeg
c1_o5_wounded.jpeg
c3_o4_still.jpeg
c2_o5_seascape.jpeg
c1_o1_view.jpeg
c2_o3_mother.jpeg
c5_o3_chaos.jpeg
sam_plaisir_jpeg.jpeg
sam_éveil_jpeg.jpeg
c5_o1_abstraction.jpeg
c4_o2_elephant.jpeg
c4_o1_brainchain.jpeg
c1_o4_sha.jpeg
c5_o6_variation.jpeg
c3_o1_joueurs.jpeg
c2_o6_conversation.jpeg
c3_o3_lake.jpeg
c3_o2_balcon.jpeg
c1_o3_portrait.jpeg


In [68]:

# metrics_TD_TSA = pd.concat([metrics ,  d],  ) 
# metrics.append(d)
metrics_TD_TDex = pd.DataFrame(columns = metrics_names)

# Compare TD <=> TDex 
for stimulus_name  in tqdm(stimuli_names):
    print(stimulus_name)
    td_fix_map_p_f = os.path.join(fixation_folder,td_folder,stimulus_name)
    td_sal_map_p_f = os.path.join(salmap_folder,td_folder,stimulus_name)
    tdex_fix_map_p_f = os.path.join(fixation_folder,tdex_folder,stimulus_name)
    tdex_sal_map_p_f = os.path.join(salmap_folder,tdex_folder,stimulus_name)

    td_fix_map_p = os.path.join(td_fix_map_p_f, [ x for x  in  os.listdir(td_fix_map_p_f) if not ("ipy" in  x) ][0]) 
    td_sal_map_p = os.path.join(td_sal_map_p_f, [ x for x  in  os.listdir(td_sal_map_p_f) if not ("ipy" in  x)][0] )
    tdex_fix_map_p = os.path.join(tdex_fix_map_p_f, [ x for x  in  os.listdir(tdex_fix_map_p_f)  if not ("ipy" in  x) ][0] )
    tdex_sal_map_p = os.path.join(tdex_sal_map_p_f, [ x for x  in  os.listdir(tdex_sal_map_p_f) if not ("ipy" in  x)][0])

    td_fix_map = cv2.imread( td_fix_map_p ,0)  
    td_sal_map = cv2.imread(td_sal_map_p ,0)
    tdex_fix_map = cv2.imread(tdex_fix_map_p ,0)
    tdex_sal_map = cv2.imread( tdex_sal_map_p,0)

    # print(tsa_fix_map.shape , tsa_sal_map.shape,  td_fix_map.shape,  td_sal_map.shape)
    metrics_row = calc_metrics(td_sal_map, td_fix_map, tdex_fix_map,tdex_sal_map)  
    metrics_row = pd.DataFrame([metrics_row])
    metrics_TD_TDex = pd.concat([metrics_TD_TDex,metrics_row], ignore_index=True)  
     

  0%|          | 0/34 [00:00<?, ?it/s]

c1_o2_civita.jpeg


/tmp/ipykernel_923639/2520252913.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_TD_TDex = pd.concat([metrics_TD_TDex,metrics_row], ignore_index=True)


c3_o6_three.jpeg
c4_o4_woman.jpeg
sam_beauté_jpeg.jpeg
c5_o5_field.jpeg
c4_o5_ventimiglia.jpeg
c2_o2_stora.jpeg
c4_o3_persistance.jpeg
c3_o5_antigraceful.jpeg
c5_o4_sails.jpeg
c4_o6_monde.jpeg
connaissance_jpeg.jpeg
c2_o1_lordship.jpeg
c5_o2_based.jpeg
c1_o6_embouchment.jpeg
c2_o4_saint_brac.jpeg
c1_o5_wounded.jpeg
c3_o4_still.jpeg
c2_o5_seascape.jpeg
c1_o1_view.jpeg
c2_o3_mother.jpeg
c5_o3_chaos.jpeg
sam_plaisir_jpeg.jpeg
sam_éveil_jpeg.jpeg
c5_o1_abstraction.jpeg
c4_o2_elephant.jpeg
c4_o1_brainchain.jpeg
c1_o4_sha.jpeg
c5_o6_variation.jpeg
c3_o1_joueurs.jpeg
c2_o6_conversation.jpeg
c3_o3_lake.jpeg
c3_o2_balcon.jpeg
c1_o3_portrait.jpeg


In [61]:
metrics_TD_TSA.to_pickle('metrics_TD_TSA.pickle')
metrics_TDex_TSA.to_pickle('metrics_TDex_TSA.pickle')

In [69]:
metrics_TD_TDex.to_pickle('metrics_TD_TDex.pickle') 

In [71]:
metrics_TD_TDex.describe() 

,KLDiv,CC,SIM,AuC,NSS,InfoGain
count,34.000000,34.000000,34.000000,0.0,34.000000,0.0
mean,20.833709,0.063115,0.034614,NaN,1.155963,NaN
std,5.882492,0.076915,0.056674,NaN,1.008474,NaN
min,10.671415,-0.002314,0.000000,NaN,-0.133326,NaN
25%,16.995078,0.001259,0.000547,NaN,0.104975,NaN
50%,20.105175,0.039843,0.017914,NaN,1.176547,NaN
75%,26.744775,0.079917,0.029524,NaN,1.802978,NaN
max,29.768912,0.274943,0.195901,NaN,3.419940,NaN


In [62]:
metrics_TD_TSA.describe() 

,KLDiv,CC,SIM,AuC,NSS,InfoGain
count,34.000000,34.000000,34.000000,0.0,34.000000,0.0
mean,19.814230,0.073468,0.032021,NaN,1.515937,NaN
std,5.678878,0.069006,0.028891,NaN,1.249779,NaN
min,6.516826,-0.000009,0.001391,NaN,-0.000251,NaN
25%,17.445249,0.018431,0.008856,NaN,0.494445,NaN
50%,20.977617,0.046884,0.023689,NaN,1.089103,NaN
75%,24.511765,0.122984,0.057060,NaN,2.567749,NaN
max,26.960415,0.213904,0.093913,NaN,4.140181,NaN


In [63]:
metrics_TDex_TSA.describe() 

,KLDiv,CC,SIM,AuC,NSS,InfoGain
count,34.000000,34.000000,34.000000,0.0,34.000000,0.0
mean,22.245821,0.056740,0.030373,NaN,1.077564,NaN
std,6.547997,0.080360,0.051394,NaN,1.111975,NaN
min,5.740121,-0.001891,0.000000,NaN,-0.143832,NaN
25%,20.811432,0.004111,0.001368,NaN,0.199663,NaN
50%,22.300644,0.025695,0.006501,NaN,0.903298,NaN
75%,27.264902,0.065080,0.026573,NaN,1.640353,NaN
max,31.661647,0.320258,0.192717,NaN,4.075273,NaN


In [64]:
metrics_TDex_TSA.mean() 

KLDiv       22.245821
CC           0.056740
SIM          0.030373
AuC               NaN
NSS          1.077564
InfoGain          NaN
dtype: float64

In [65]:
metrics_TD_TSA.mean() 

KLDiv       19.814230
CC           0.073468
SIM          0.032021
AuC               NaN
NSS          1.515937
InfoGain          NaN
dtype: float64

In [ ]:
os.listdir('exp-et/data_organized/TDex/c1_o2_civita.jpeg/BEAT681_block2.csv') 

In [37]:
import pandas as pd

# Create an initial DataFrame
data = {
    'A': [1, 2],
    'B': [3, 4]
}
df = pd.DataFrame(data)

# Create a dictionary to add as a new row
new_row = {'A': 5, 'B': 6}

# Convert the dictionary to a DataFrame
new_row_df = pd.DataFrame([new_row])

# Concatenate the new row DataFrame with the original DataFrame
df = pd.concat([df, new_row_df], ignore_index=True)

print(df)

   A  B
0  1  3
1  2  4
2  5  6


In [15]:

gt_fix_map_p = "./fix_maps/TD/c1_o1_view.jpeg/ANMT322_bloc1_fixation_map.png"
gt_sal_map_p = "./salmaps/TD/c1_o1_view.jpeg/ANMT322_bloc1_saliency_map.png"

pred_fix_p = "./fix_maps/TSA/c1_o1_view.jpeg/QUPP719_bloc1_fixation_map.png"
pred_sal_p = "./salmaps/TSA/c1_o1_view.jpeg/QUPP719_bloc1_saliency_map.png"


# this is just the name its not actually discretised (binary)
gt_sal = cv2.imread(gt_sal_map_p,0)
gt_fix = cv2.imread(gt_fix_map_p,0)



s_map = cv2.imread(pred_sal_p,0)
s_map_norm = normalize_map(s_map)


calc_metrics(s_map, None, gt_sal, gt_fix) 

# auc_judd_score = auc_judd(s_map_norm,gt_fix)
# print('auc judd :', auc_judd_score)
# # auc_borji_score = auc_borji(s_map_norm,gt_fix)
# # print('auc borji :', auc_borji_score)
# # auc_shuff_score = auc_shuff(s_map_norm,gt,gt)
# # print 'auc shuffled :', auc_shuff_score

# nss_score = nss(s_map,gt_fix)
# print('nss :', nss_score)
# # infogain_score = infogain(s_map_norm,gt,gt)
# # print 'info gain :', infogain_score



# #continous gts
# sim_score = similarity(s_map,gt_sal)
# print('sim score :', sim_score)
# cc_score = cc(s_map,gt_sal)
# print('cc score :',cc_score)
# kldiv_score = kldiv(s_map,gt_sal)
# print('kldiv score :',kldiv_score)


/tmp/ipykernel_923639/509843620.py:32: UserWarning: can improve the way GT is discretized
  warnings.warn('can improve the way GT is discretized')


{'KLDiv': np.float64(20.938411147541828),
 'CC': np.float64(0.04203653084922859),
 'SIM': np.float64(0.05702712726587408),
 'AuC': nan,
 'NSS': np.float64(-0.10469449755763655),
 'InfoGain': nan}

In [66]:
stimuli 

['c1_o2_civita.jpeg',
 'c3_o6_three.jpeg',
 'c4_o4_woman.jpeg',
 'sam_beauté_jpeg.jpeg',
 'c5_o5_field.jpeg',
 'c4_o5_ventimiglia.jpeg',
 'c2_o2_stora.jpeg',
 'c4_o3_persistance.jpeg',
 'c3_o5_antigraceful.jpeg',
 'c5_o4_sails.jpeg',
 'c4_o6_monde.jpeg',
 'connaissance_jpeg.jpeg',
 'c2_o1_lordship.jpeg',
 'c5_o2_based.jpeg',
 'c1_o6_embouchment.jpeg',
 'c2_o4_saint_brac.jpeg',
 'c1_o5_wounded.jpeg',
 'c3_o4_still.jpeg',
 'c2_o5_seascape.jpeg',
 'c1_o1_view.jpeg',
 'c2_o3_mother.jpeg',
 'c5_o3_chaos.jpeg',
 'sam_plaisir_jpeg.jpeg',
 'sam_éveil_jpeg.jpeg',
 'c5_o1_abstraction.jpeg',
 'c4_o2_elephant.jpeg',
 'c4_o1_brainchain.jpeg',
 'c1_o4_sha.jpeg',
 'c5_o6_variation.jpeg',
 'c3_o1_joueurs.jpeg',
 'c2_o6_conversation.jpeg',
 'c3_o3_lake.jpeg',
 'c3_o2_balcon.jpeg',
 'c1_o3_portrait.jpeg']